In [1]:
import pandas as pd 
import numpy as np 
import sys
import asyncio
sys.path.append('../scripts')

from telegram_scraper import scrape_telegram_channels

In [ ]:
# Load all secretes 

from config import API_HOST, API_PORT, POSTGRES_USER

print(f"Connecting to DB as user: {POSTGRES_USER}")


## Data Scraping and Collection (Extract & Load)
Extract relevant text, image, price, product name, and timestamp data from public Telegram channels and save it to your raw data zone (e.g., JSON or Parquet files).

### Scraping Telegram data using json format

In [2]:
# Run the async scraper function from notebook
await scrape_telegram_channels()

Scraping: https://t.me/Chemed123
Scraping: https://t.me/lobelia4cosmetics
Scraping: https://t.me/tikvahpharma
✅ Saved 2071 messages to data/raw/telegram_messages.json


### Scraping Telegram image data 